<a href="https://colab.research.google.com/github/atick-faisal/Hand-Gesture-Recognition/blob/main/Spatial-Projection-Analysis/VHSpatialProjectionTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import time
import joblib
import shutil
import tarfile
import requests
import gdown
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score

tf.__version__

'2.5.0'

In [2]:
TEST_USER    = '004'

DATASET_ID   = '1p0CSRb9gax0sKqdyzOYVt-BXvZ4GtrBv'

# -------------BASE DIR (MODIFY THIS TO YOUR NEED) ------------ #
BASE_DIR     = os.getcwd()
# BASE_DIR     = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'

DATA_DIR     = 'Sensor-Data/'
BW_IMG_DIR   = 'BW-Spatial-Path-Images/'
RGB_IMG_DIR  = 'RGB-Spatial-Path-Images/'
IMG_SIZE     = (3, 3) # INCHES

IMG_DIR      = 'BW-Spatial-Path-Images/'
LOG_DIR      = 'Logs/'

USERS        = ['001', '002', '003', '004', '005', '006', '007', '008', '009',
                '010', '011', '012', '013', '014', '015', '016', '017', '018',
                '019', '020', '021', '022', '023', '024', '025']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES     = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES       = ['XY', 'YZ', 'ZX', 'VH']

DT           = 0.01
LINEWIDTH    = 7

BATCH_SIZE   = 32
IMG_LEN      = 48
IMG_SIZE     = (IMG_LEN, IMG_LEN)
CHANNELS     = 3
IMG_SHAPE    = IMG_SIZE + (CHANNELS,)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN  = 4000
TRAIN_LEN    = 3840
TEST_LEN     = 160

EPOCHS       = 15
LR           = 0.001
DECAY        = 0.0

In [3]:
#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')   

In [4]:
# ------- Comment This if already downloaded -------- #

# destination = os.path.join(BASE_DIR, DATA_DIR)
# download_data(DATASET_ID, destination)

In [5]:
# ------------- Spatial Path Image Generation ----------- #

def clean_dir(path):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(path)
        print('√')
    except:
        print('✕')
    
    print('creating ' + path + ' directory ... ', end='')
    os.mkdir(path)
    print('√')
    
def get_user_acceleration_angle(gx, gy, gz, lx, ly, lz):

    # Calculates angle between linear acceleration and gravity
    # Argumnet(s):
    #    - gx, gy, gz : Gravity vector components
    #    - lx, ly, lz : Linear acceleration components

    n = gx * lx + gy * ly + gz * lz
    d = np.sqrt(gx ** 2 + gy ** 2 + gz ** 2) * np.sqrt(lx ** 2 + ly ** 2 + lz ** 2)

    # Division by zero is not allowed
    # Replacing all 0s with 0.000001
    #d = np.where(d == 0, 0.000001, d)

    theta = np.arccos(np.divide(n, d))
    
    return np.nan_to_num(theta)
    
# ----------- Spatial Path Vector Calculation ----------- #

def get_displacement(acc):
    v = np.zeros(acc.shape)
    d = np.zeros(acc.shape)
    for i in range(acc.shape[0] - 1):
        v[i + 1] = v[i] + acc[i] * DT
        d[i + 1] = v[i] * DT + 0.5 * acc[i] * DT * DT
        
    return d

def write_image(x, y, path):
    fig, ax = plt.subplots(frameon=True, figsize=(3, 3))
    ax.axis('off')
    plt.plot(x, y, '-k', linewidth=LINEWIDTH)
    plt.savefig(path)
    plt.close()

def generate_bw_images():
    count = 0
    image_dir = os.path.join(BASE_DIR, BW_IMG_DIR)
    clean_dir(image_dir)
    
    for plane in PLANES:
        print('processing spatial path images for ' + plane + ' plane ... ', end='')
        plane_dir = os.path.join(image_dir, plane)
        os.mkdir(plane_dir)
        
        for gesture in GESTURES:
            os.mkdir(os.path.join(plane_dir, gesture))
    
            for user in USERS:
                user_dir = os.path.join(BASE_DIR, DATA_DIR, user)
                gesture_dir = os.path.join(user_dir, gesture + '.csv')
                
                accx = pd.read_csv(gesture_dir)['ACCx'].to_numpy()
                accy = pd.read_csv(gesture_dir)['ACCy'].to_numpy()
                accz = pd.read_csv(gesture_dir)['ACCz'].to_numpy()
                
                lacx = pd.read_csv(gesture_dir)['ACCx_body'].to_numpy()
                lacy = pd.read_csv(gesture_dir)['ACCy_body'].to_numpy()
                lacz = pd.read_csv(gesture_dir)['ACCz_body'].to_numpy()
                
                grax = accx - lacx
                gray = accy - lacy
                graz = accz - lacz
                
                theta = get_user_acceleration_angle(grax, gray, graz, lacx, lacy, lacz)
                lm = np.sqrt(lacx ** 2 + lacy ** 2 + lacz ** 2)

                accv = lm * np.cos(theta)
                acch = lm * np.sin(theta)

                x = get_displacement(accx).reshape(-1, 150)
                y = get_displacement(accy).reshape(-1, 150)
                z = get_displacement(accz).reshape(-1, 150)
                v = get_displacement(accv).reshape(-1, 150)
                h = get_displacement(acch).reshape(-1, 150)

                for i in range(x.shape[0]):
                    image_name = 'u' + user + '_g' + '{:0>2d}'.format(GESTURES.index(gesture)) + \
                                 '_s' + '{:0>7d}'.format(count) + '_p' + plane + '.jpg'
                    
                    path = os.path.join(BASE_DIR, BW_IMG_DIR, plane, gesture, image_name)
                    
                    if plane == 'XY':
                        write_image(x[i, :], y[i, :], path)
                    elif plane == 'YZ':
                        write_image(y[i, :], z[i, :], path)
                    elif plane == 'ZX':
                        write_image(z[i, :], x[i, :], path)
                    else:
                        write_image(h[i, :], v[i, :], path)

                    count = count + 1
            
        print('√')

In [6]:
# generate_bw_images()

In [7]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, CHANNELS))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, CHANNELS))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(
                os.path.join(path, filename),
                cv2.IMREAD_GRAYSCALE
            )
            resized = cv2.resize(img, IMG_SIZE)

            if resized.ndim < 3:
                resized = np.expand_dims(resized, axis=-1)
                
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, CHANNELS))
    y = np.zeros((DATASET_LEN, 1))
    
    train_count = 0
    test_count  = TRAIN_LEN
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X[train_count, :] = resized
                y[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X[test_count, :] = resized
                y[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_train = X[:TRAIN_LEN, :, :, :]
    y_train = y[:TRAIN_LEN, :]
    X_test = X[TRAIN_LEN:, :, :, :]
    y_test = y[TRAIN_LEN:, :]

    print('√')

    return X_train, X_test, y_train, y_test

In [8]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.3),
])
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [9]:
# def get_model():
#     inputs = tf.keras.Input(shape=IMG_SHAPE)
#     x = data_augmentation(inputs)
#     x = preprocess_input(inputs)
#     x = base_model(x, training=False)
#     x = global_average_layer(x)
#     x = tf.keras.layers.Dropout(0.2)(x)
#     outputs = prediction_layer(x)
#     model = tf.keras.Model(inputs, outputs)
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
#     return model

def get_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=IMG_SHAPE))
    model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255.0, offset=0))
    model.add(tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.0))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.0))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Dropout(0.0))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(GESTURES)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    model.summary()

    return model

In [10]:
model_xy = get_model()
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 16)       

In [11]:
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

del X_train_xy, X_test_xy, y_train_xy, y_test_xy

              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95        10
         1.0       1.00      1.00      1.00        10
         2.0       0.00      0.00      0.00        10
         3.0       0.00      0.00      0.00        10
         4.0       0.00      0.00      0.00        10
         5.0       0.00      0.00      0.00        10
         6.0       1.00      0.10      0.18        10
         7.0       0.29      0.20      0.24        10
         8.0       0.00      0.00      0.00        10
         9.0       0.33      0.70      0.45        10
        10.0       0.00      0.00      0.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       1.00      0.10      0.18        10
        14.0       0.43      0.30      0.35        10
        15.0       0.12      1.00      0.22        10

    accuracy                           0.28       160
   macro avg       0.32   

In [12]:
model_yz = get_model()
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 16)        0

In [13]:
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

del X_train_yz, X_test_yz, y_train_yz, y_test_yz

              precision    recall  f1-score   support

         0.0       0.55      0.60      0.57        10
         1.0       0.56      1.00      0.71        10
         2.0       0.00      0.00      0.00        10
         3.0       0.00      0.00      0.00        10
         4.0       0.00      0.00      0.00        10
         5.0       1.00      0.10      0.18        10
         6.0       0.15      0.20      0.17        10
         7.0       0.43      0.30      0.35        10
         8.0       0.00      0.00      0.00        10
         9.0       0.32      0.90      0.47        10
        10.0       0.00      0.00      0.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.06      0.10      0.08        10
        13.0       0.71      1.00      0.83        10
        14.0       0.77      1.00      0.87        10
        15.0       0.15      0.40      0.22        10

    accuracy                           0.35       160
   macro avg       0.29   

In [14]:
model_zx = get_model()
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')
X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 16)        0

In [15]:
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

del X_train_zx, X_test_zx, y_train_zx

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        10
         1.0       1.00      1.00      1.00        10
         2.0       1.00      0.10      0.18        10
         3.0       0.00      0.00      0.00        10
         4.0       0.00      0.00      0.00        10
         5.0       0.50      0.20      0.29        10
         6.0       0.60      0.30      0.40        10
         7.0       0.25      0.10      0.14        10
         8.0       0.00      0.00      0.00        10
         9.0       0.02      0.10      0.03        10
        10.0       0.00      0.00      0.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       0.00      0.00      0.00        10
        14.0       0.48      1.00      0.65        10
        15.0       0.37      0.70      0.48        10

    accuracy                           0.22       160
   macro avg       0.26   

In [16]:
model_vh = get_model()
X_train_vh, X_test_vh, y_train_vh, y_test_vh = load_data('VH')
X_train_vh, y_train_vh = shuffle(X_train_vh, y_train_vh)
history_vh = model_vh.fit(X_train_vh, y_train_vh, validation_data=(X_test_vh, y_test_vh), epochs=EPOCHS)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 48, 48, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 16)        0

In [17]:
y_pred_vh = model_vh.predict(X_test_vh)
y_pred = np.argmax(y_pred_vh, axis=1)
print(classification_report(y_test_vh.ravel(), y_pred, zero_division=0))
prc_vh = precision_score(y_test_vh.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

del X_train_vh, X_test_vh, y_train_vh

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        10
         2.0       0.57      0.80      0.67        10
         3.0       0.00      0.00      0.00        10
         4.0       0.00      0.00      0.00        10
         5.0       0.45      0.50      0.48        10
         6.0       1.00      0.20      0.33        10
         7.0       0.56      0.50      0.53        10
         8.0       0.00      0.00      0.00        10
         9.0       0.53      1.00      0.69        10
        10.0       0.00      0.00      0.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.33      0.10      0.15        10
        13.0       0.75      0.30      0.43        10
        14.0       0.00      0.00      0.00        10
        15.0       0.16      1.00      0.28        10

    accuracy                           0.40       160
   macro avg       0.40   

In [18]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx + y_pred_vh
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_zx.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        10
         2.0       0.00      0.00      0.00        10
         3.0       0.00      0.00      0.00        10
         4.0       0.00      0.00      0.00        10
         5.0       0.14      0.10      0.12        10
         6.0       1.00      0.90      0.95        10
         7.0       0.80      0.40      0.53        10
         8.0       0.00      0.00      0.00        10
         9.0       0.31      1.00      0.48        10
        10.0       0.00      0.00      0.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       1.00      0.40      0.57        10
        14.0       1.00      1.00      1.00        10
        15.0       0.14      1.00      0.24        10

    accuracy                           0.42       160
   macro avg       0.40   